# How to Extract the Lyrics of Taylor Swift's Songs From the Genius API

I followed this tutorial from Melanie Walsh's Intro to Cultural Analytics & Python book:

https://melaniewalsh.github.io/Intro-Cultural-Analytics/04-Data-Collection/08-Collect-Genius-Lyrics.html

## Step 1: Connect to Genius

In [1]:
from bs4 import BeautifulSoup
import re
import lyricsgenius
import requests
from pathlib import Path
import pandas as pd

def clean_up(song_title):

    if "Ft" in song_title:
        before_ft_pattern = re.compile(".*(?=\(Ft)")
        song_title_before_ft = before_ft_pattern.search(song_title).group(0)
        clean_song_title = song_title_before_ft.strip()
        clean_song_title = clean_song_title.replace("/", "-")
    
    else:
        song_title_no_lyrics = song_title.replace("Lyrics", "")
        clean_song_title = song_title_no_lyrics.strip()
        clean_song_title = clean_song_title.replace("/", "-")
    
    return clean_song_title

def get_all_songs_from_album(artist, album_name):
    
    artist = artist.replace(" ", "-")
    album_name = album_name.replace(" ", "-")
    
    response = requests.get(f"https://genius.com/albums/{artist}/{album_name}")
    html_string = response.text
    document = BeautifulSoup(html_string, "html.parser")
    song_title_tags = document.find_all("h3", attrs={"class": "chart_row-content-title"})
    song_titles = [song_title.text for song_title in song_title_tags]
    
    clean_songs = []
    for song_title in song_titles:
        clean_song = clean_up(song_title)
        clean_songs.append(clean_song)
        
    return clean_songs

def download_album_lyrics(artist, album_name): 
    
    # You will need to go to Genius Developers to get your own client access token
    client_access_token = 'insert_your_client_access_token_here'
    LyricsGenius = lyricsgenius.Genius(client_access_token)
    LyricsGenius.remove_section_headers = True
    
    clean_songs = get_all_songs_from_album(artist, album_name)
    
    for song in clean_songs:
        
        song_object = LyricsGenius.search_song(song, artist)
        
        if song_object != None:
            
            artist_title = artist.replace(" ", "-")
            album_title = album_name.replace(" ", "-")
            song_title = song.replace("/", "-")
            song_title = song.replace(" ", "-")
            
            custom_filename=f"{artist_title}_{album_title}/{song_title}"
            

            Path(f"{artist_title}_{album_title}").mkdir(parents=True, exist_ok=True)
            
            song_object.save_lyrics(filename=custom_filename, extension='txt', sanitize=False)
        
        else:
            print('No lyrics')

## Step 2: Specify Your Artists + Albums of Interest

In [2]:
# I ran a few lines at a time to not get a timeout error from Genius

#download_album_lyrics("Taylor Swift", "Taylor Swift")
#download_album_lyrics("Taylor Swift", "Fearless")
#download_album_lyrics("Taylor Swift", "Speak Now")
#download_album_lyrics("Taylor Swift", "Red")
#download_album_lyrics("Taylor Swift", "1989")
#download_album_lyrics("Taylor Swift", "Reputation")
#download_album_lyrics("Taylor Swift", "Lover")
#download_album_lyrics("Taylor Swift", "folklore")
#download_album_lyrics("Taylor Swift", "evermore")
#download_album_lyrics("Taylor Swift", "Midnights")

## Step 3: Load Text Data into a Clean DataFrame

In [3]:
import re
import string

def clean_text(text):
    
    # Some light data cleaning - you will need to adjust based on your data
    text = text.replace('See Taylor Swift LiveGet tickets as low as $270', ' ') # remove ad
    text = re.sub('\d*Embed', ' ', text) # remove ending text with number + Embed
    
    return text

In [4]:
# Specify the folder names with the lyric data from Genius
directory_paths = ['Taylor-Swift_Taylor-Swift/',
                   'Taylor-Swift_Fearless/',
                   'Taylor-Swift_Speak-Now/',
                   'Taylor-Swift_Red/',
                   'Taylor-Swift_1989/',
                   'Taylor-Swift_Reputation/',
                   'Taylor-Swift_Lover/',
                   'Taylor-Swift_folklore/',
                   'Taylor-Swift_evermore/',
                   'Taylor-Swift_Midnights/']

In [5]:
pd.options.display.max_rows = 500
pd.set_option('display.max_colwidth', 0)

ts_lyrics = pd.DataFrame({"Album": [],
                          "Song Name": [],
                          "Lyrics": []})

idx = 0

for i, album in enumerate(directory_paths):
    
    album_name = album[13:-1].replace("-", " ")
        
    for song in Path(album).glob('*.txt'):
        
        song_name = str(song).replace("-", " ").split("/")[1][:-4]
                
        full_text = open(song, encoding="utf-8")
        lyrics_list = full_text.readlines()[1:] #read()
        lyrics_list = [l.replace("\n", "") for l in lyrics_list]
        lyrics = ' '.join(lyrics_list)
        lyrics = clean_text(lyrics)
        full_text.close()
        
        ts_lyrics.loc[idx] = [album_name, song_name, lyrics]
        idx += 1

In [6]:
ts_lyrics

,Album,Song Name,Lyrics
0,Taylor Swift,Mary's Song (Oh My My My),"She said I was seven and you were nine I looked at you like the stars that shined In the sky, the pretty lights And our daddies used to joke about the two of us Growing up and falling in love And our mamas smiled and rolled their eyes And said, ""Oh, my, my, my"" Take me back to the house in the backyard tree Said you'd beat me up, you were bigger than me You never did, you never did Take me back when our world was one block wide I dared you to kiss me and ran when you tried Just two kids, you and I Oh, my, my, my, my Well, I was sixteen when suddenly I wasn't that little girl you used to see But your eyes still shined like pretty lights And our daddies used to joke about the two of us They never believed we'd really fall in love And our mamas smiled and rolled their eyes And said, ""Oh, my, my, my"" Take me back to the creek beds we turned up 2 a.m. riding in your truck And all I need is you next to me Take me back to the time we had our very first fight The slamming of doors 'stead of kissing goodnight You stayed outside 'til the morning light Oh, my, my, my, my A few years had gone and come around We were sitting at our favorite spot in town And you looked at me, got down on one knee Take me back to the time when we walked down the aisle Our whole town came and our mamas cried You said, ""I do,"" and I did, too Take me home where we met so many years before We'll rock our babies on that very front porch After all this time, you and I You might also like And I'll be eighty-seven, you'll be eighty-nine I'll still look at you like the stars that shine In the sky, oh, my, my, my"
1,Taylor Swift,A Perfectly Good Heart,"Why would you wanna break A perfectly good heart? Why would you wanna take Our love and tear it all apart now? Why would you wanna make The very first scar? Why would you wanna break A perfectly good heart? Maybe I should've seen the signs Should've read the writing on the wall And realized by the distance in your eyes That I would be the one to fall No matter what you say I still can't believe that you would walk away It don't make sense to me, but Why would you wanna break A perfectly good heart? Why would you wanna take Our love and tear it all apart now? Why would you wanna make The very first scar? Why would you wanna break A perfectly good heart? It's not unbroken anymore (It's not unbroken anymore) How do I get it back the way it was before? Why would you wanna break A perfectly good heart? Why would you wanna take Our love and tear it all apart now? Why would you wanna make The very first scar? Why would you wanna break— (Why) Would you wanna break it? You might also like Why would you wanna break A perfectly good heart? Why would you wanna take Our love and tear it all apart now? Why would you wanna make The very first scar? Why would you wanna break A perfectly good heart? Mmm-hmm"
2,Taylor Swift,Tim McGraw,"He said the way my blue eyes shined Put those Georgia stars to shame that night I said, ""That's a lie"" Just a boy in a Chevy truck That had a tendency of gettin' stuck On backroads at night And I was right there beside him all summer long And then the time we woke up to find that summer gone But when you think Tim McGraw I hope you think my favorite song The one we danced to all night long The moon like a spotlight on the lake When you think happiness I hope you think that little black dress Think of my head on your chest And my old faded blue jeans When you think Tim McGraw I hope you think of me September saw a month of tears And thankin' God that you weren't here To see me like that But in a box beneath my bed Is a letter that you never read From three summers back It's hard not to find it all a little bittersweet And lookin' back on all of that, it's nice to believe When you think Tim McGraw I hope you think my favorite song The one we danced to all night long The moon like a spotlight on the lake When you think happiness I hope 

In [7]:
# Remove 8 extra rows of data including duplicate songs, deluxe songs and non-songs
# to match the same 147 songs that were on the Spotify API list

ts_lyrics.drop(13, axis=0, inplace=True) # Teardrops on My Guitar (Pop Version)
ts_lyrics.drop(44, axis=0, inplace=True) # The Moment I Knew
ts_lyrics.drop(49, axis=0, inplace=True) # Come Back... Be Here
ts_lyrics.drop(55, axis=0, inplace=True) # Girl at Home
ts_lyrics.drop(75, axis=0, inplace=True) # Reputation Magazine Vol. 1
ts_lyrics.drop(77, axis=0, inplace=True) # Why She Disappeared [Poem]
ts_lyrics.drop(86, axis=0, inplace=True) # Reputation [Prologue]
ts_lyrics.drop(88, axis=0, inplace=True) # If You're Anything Like Me [Poem]
ts_lyrics.reset_index(inplace=True)

In [8]:
ts_lyrics

,index,Album,Song Name,Lyrics
0,0,Taylor Swift,Mary's Song (Oh My My My),"She said I was seven and you were nine I looked at you like the stars that shined In the sky, the pretty lights And our daddies used to joke about the two of us Growing up and falling in love And our mamas smiled and rolled their eyes And said, ""Oh, my, my, my"" Take me back to the house in the backyard tree Said you'd beat me up, you were bigger than me You never did, you never did Take me back when our world was one block wide I dared you to kiss me and ran when you tried Just two kids, you and I Oh, my, my, my, my Well, I was sixteen when suddenly I wasn't that little girl you used to see But your eyes still shined like pretty lights And our daddies used to joke about the two of us They never believed we'd really fall in love And our mamas smiled and rolled their eyes And said, ""Oh, my, my, my"" Take me back to the creek beds we turned up 2 a.m. riding in your truck And all I need is you next to me Take me back to the time we had our very first fight The slamming of doors 'stead of kissing goodnight You stayed outside 'til the morning light Oh, my, my, my, my A few years had gone and come around We were sitting at our favorite spot in town And you looked at me, got down on one knee Take me back to the time when we walked down the aisle Our whole town came and our mamas cried You said, ""I do,"" and I did, too Take me home where we met so many years before We'll rock our babies on that very front porch After all this time, you and I You might also like And I'll be eighty-seven, you'll be eighty-nine I'll still look at you like the stars that shine In the sky, oh, my, my, my"
1,1,Taylor Swift,A Perfectly Good Heart,"Why would you wanna break A perfectly good heart? Why would you wanna take Our love and tear it all apart now? Why would you wanna make The very first scar? Why would you wanna break A perfectly good heart? Maybe I should've seen the signs Should've read the writing on the wall And realized by the distance in your eyes That I would be the one to fall No matter what you say I still can't believe that you would walk away It don't make sense to me, but Why would you wanna break A perfectly good heart? Why would you wanna take Our love and tear it all apart now? Why would you wanna make The very first scar? Why would you wanna break A perfectly good heart? It's not unbroken anymore (It's not unbroken anymore) How do I get it back the way it was before? Why would you wanna break A perfectly good heart? Why would you wanna take Our love and tear it all apart now? Why would you wanna make The very first scar? Why would you wanna break— (Why) Would you wanna break it? You might also like Why would you wanna break A perfectly good heart? Why would you wanna take Our love and tear it all apart now? Why would you wanna make The very first scar? Why would you wanna break A perfectly good heart? Mmm-hmm"
2,2,Taylor Swift,Tim McGraw,"He said the way my blue eyes shined Put those Georgia stars to shame that night I said, ""That's a lie"" Just a boy in a Chevy truck That had a tendency of gettin' stuck On backroads at night And I was right there beside him all summer long And then the time we woke up to find that summer gone But when you think Tim McGraw I hope you think my favorite song The one we danced to all night long The moon like a spotlight on the lake When you think happiness I hope you think that little black dress Think of my head on your chest And my old faded blue jeans When you think Tim McGraw I hope you think of me September saw a month of tears And thankin' God that you weren't here To see me like that But in a box beneath my bed Is a letter that you never read From three summers back It's hard not to find it all a little bittersweet And lookin' back on all of that, it's nice to believe When you think Tim McGraw I hope you think my favorite song The one we danced to all night long The moon like a spotlight on the lake When you think happi

## Step 4: Export the Lyrics as a CSV File

In [9]:
ts_lyrics.to_csv("taylor_swift_genius_data.csv", index=False)